In [1]:
import pandas as pd
import numpy as np
import bs4 as bs
import urllib.request

In [2]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2021"

In [3]:
source = urllib.request.urlopen(link).read()
soup = bs.BeautifulSoup(source,'lxml')

In [4]:
tables = soup.find_all('table',class_='wikitable sortable')

In [5]:
len(tables)

4

In [6]:
df1 = pd.read_html(str(tables[0]))[0]
df2 = pd.read_html(str(tables[1]))[0]
df3 = pd.read_html(str(tables[2]))[0]
df4 = pd.read_html(str(tables[3]).replace("'1\"\'",'"1"'))

In [7]:
df = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)

In [8]:
df

,Opening,Opening.1,Title,Production company,Cast and crew,.mw-parser-output .tooltip-dotted{border-bottom:1px dotted;cursor:help}Ref.,Ref.
0,JANUARY,1.0,Shadow in the Cloud,Vertical Entertainment,Roseanne Liang (director/screenplay); Max Land...,[2],NaN
1,JANUARY,13.0,The White Tiger,Netflix,Ramin Bahrani (director/screenplay); Adarsh Go...,NaN,NaN
2,JANUARY,14.0,Locked Down,HBO Max / Warner Bros. Pictures,Doug Liman (director); Steven Knight (screenpl...,[3],NaN
3,JANUARY,15.0,The Dig,Netflix / Clerkenwell Films,Simon Stone (director); Moira Buffini (screenp...,[4],NaN
4,JANUARY,15.0,Outside the Wire,Netflix,"Mikael Håfström (director); Rob Yescombe, Rowa...",[5],NaN
...,...,...,...,...,...,...,...
356,DECEMBER,25.0,The Tragedy of Macbeth,Apple TV+ / A24 / IAC Films,Joel Coen (director/screenplay); Denzel Washin...,NaN,[273]
357,DECEMBER,25.0,A Journal for Jordan,Columbia Pictures / Escape Artists / Bron Studios,Denzel Washington (director); Virgil Williams ...,NaN,[274]
358,DECEMBER,25.0,American Underdog,Lionsgate,"Erwin brothers (directors); Jon Erwin, David A...",NaN,[275]
359,DECEMBER,26.0,Memoria,Neon,Apichatpong Weerasethakul (director/acreenplay...,NaN,[276]


In [9]:
df_2021 = df[['Title']]

In [10]:
df_2021

,Title
0,Shadow in the Cloud
1,The White Tiger
2,Locked Down
3,The Dig
4,Outside the Wire
...,...
356,The Tragedy of Macbeth
357,A Journal for Jordan
358,American Underdog
359,Memoria


In [11]:
!pip install tmdbv3api

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [12]:
from tmdbv3api import TMDb
import json
tmdb = TMDb()
tmdb.api_key = '2d71ac9f8f76e2fe996cdc92cab9f9cd'

In [13]:
from tmdbv3api import Movie
import json
tmdb_movie = Movie() 

In [14]:
def ids(x):
  result = tmdb_movie.search(x)
  if not result:
    return np.NaN
  else:
    return int(result[0].id)

In [15]:
df_2021['id'] = df_2021['Title'].map(lambda x: ids(str(x)))

<ipython-input-15-1e7fcbae526d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2021['id'] = df_2021['Title'].map(lambda x: ids(str(x)))


In [16]:
df_2021

,Title,id
0,Shadow in the Cloud,675327
1,The White Tiger,628534
2,Locked Down,741228
3,The Dig,532865
4,Outside the Wire,775996
...,...,...
356,The Tragedy of Macbeth,591538
357,A Journal for Jordan,501841
358,American Underdog,673309
359,Memoria,511819


In [17]:
df_2021[df_2021['id'].isnull()]

,Title,id


In [19]:
df_2021['id'] = df_2021['id'].astype('int')

<ipython-input-19-152f78e33417>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2021['id'] = df_2021['id'].astype('int')


In [20]:
def description(movie_id):
        description =""
    
        movie_details = tmdb_movie.details(movie_id)
        if movie_details['overview'] and movie_details['tagline']:
          return movie_details['overview'] + "@#$" + movie_details['tagline']
        elif movie_details['overview'] and not movie_details['tagline']:
          return movie_details['overview'] + "@#$" + ""
        elif not movie_details['overview'] and movie_details['tagline']:
          return ""+ "@#$" + movie_details['tagline']
        else:
          return "@#$"

In [21]:
df_2021['description'] = df_2021['id'].map(lambda x: description(int(x)))

<ipython-input-21-d1644edc4ea9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2021['description'] = df_2021['id'].map(lambda x: description(int(x)))


In [22]:
df_2021

,Title,id,description
0,Shadow in the Cloud,675327,A WWII pilot traveling with top secret documen...
1,The White Tiger,628534,An ambitious Indian driver uses his wit and cu...
2,Locked Down,741228,"During a COVID-19 lockdown, sparring couple Li..."
3,The Dig,532865,"As WWII looms, a wealthy widow hires an amateu..."
4,Outside the Wire,775996,"In the near future, a drone pilot is sent into..."
...,...,...,...
356,The Tragedy of Macbeth,591538,"Macbeth, the Thane of Glamis, receives a proph..."
357,A Journal for Jordan,501841,Based on the true story of First Sergeant Char...
358,American Underdog,673309,"The true story of Kurt Warner, who went from a..."
359,Memoria,511819,A Scottish orchid farmer visiting her ill sist...


In [23]:
df_2021['description'] = df_2021['description'].apply(lambda x: x.split("@#$"))

<ipython-input-23-3d8373d058b3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2021['description'] = df_2021['description'].apply(lambda x: x.split("@#$"))


In [24]:
df_2021

,Title,id,description
0,Shadow in the Cloud,675327,[A WWII pilot traveling with top secret docume...
1,The White Tiger,628534,[An ambitious Indian driver uses his wit and c...
2,Locked Down,741228,"[During a COVID-19 lockdown, sparring couple L..."
3,The Dig,532865,"[As WWII looms, a wealthy widow hires an amate..."
4,Outside the Wire,775996,"[In the near future, a drone pilot is sent int..."
...,...,...,...
356,The Tragedy of Macbeth,591538,"[Macbeth, the Thane of Glamis, receives a prop..."
357,A Journal for Jordan,501841,[Based on the true story of First Sergeant Cha...
358,American Underdog,673309,"[The true story of Kurt Warner, who went from ..."
359,Memoria,511819,[A Scottish orchid farmer visiting her ill sis...


In [25]:
df_2021['overview'] = df_2021['description'].map(lambda x: x[0])
df_2021['tagline'] = df_2021['description'].map(lambda x: x[1])

<ipython-input-25-401310be8d95>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2021['overview'] = df_2021['description'].map(lambda x: x[0])
<ipython-input-25-401310be8d95>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2021['tagline'] = df_2021['description'].map(lambda x: x[1])


In [26]:
df_2021

,Title,id,description,overview,tagline
0,Shadow in the Cloud,675327,[A WWII pilot traveling with top secret docume...,A WWII pilot traveling with top secret documen...,Every mission has its demons.
1,The White Tiger,628534,[An ambitious Indian driver uses his wit and c...,An ambitious Indian driver uses his wit and cu...,Eat or get eaten up.
2,Locked Down,741228,"[During a COVID-19 lockdown, sparring couple L...","During a COVID-19 lockdown, sparring couple Li...",Some people unravel faster than others
3,The Dig,532865,"[As WWII looms, a wealthy widow hires an amate...","As WWII looms, a wealthy widow hires an amateu...",Nothing stays lost forever.
4,Outside the Wire,775996,"[In the near future, a drone pilot is sent int...","In the near future, a drone pilot is sent into...",Defiant by design.
...,...,...,...,...,...
356,The Tragedy of Macbeth,591538,"[Macbeth, the Thane of Glamis, receives a prop...","Macbeth, the Thane of Glamis, receives a proph...",
357,A Journal for Jordan,501841,[Based on the true story of First Sergeant Cha...,Based on the true story of First Sergeant Char...,
358,American Underdog,673309,"[The true story of Kurt Warner, who went from ...","The true story of Kurt Warner, who went from a...",Some dreams become destiny.
359,Memoria,511819,[A Scottish orchid farmer visiting her ill sis...,A Scottish orchid farmer visiting her ill sist...,"In here, time stops."


In [27]:
df_2021['comb'] = df_2021['overview'] + df_2021['tagline']
df_2021['comb'] = df_2021['comb'].fillna('')

<ipython-input-27-019fde0cf68e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2021['comb'] = df_2021['overview'] + df_2021['tagline']
<ipython-input-27-019fde0cf68e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2021['comb'] = df_2021['comb'].fillna('')


In [28]:
df_2021 = df_2021.rename(columns={'Title':'title'})

In [29]:
df_2021 = df_2021.loc[:,['id','overview','tagline','title','comb']]

In [30]:
df_2021

,id,overview,tagline,title,comb
0,675327,A WWII pilot traveling with top secret documen...,Every mission has its demons.,Shadow in the Cloud,A WWII pilot traveling with top secret documen...
1,628534,An ambitious Indian driver uses his wit and cu...,Eat or get eaten up.,The White Tiger,An ambitious Indian driver uses his wit and cu...
2,741228,"During a COVID-19 lockdown, sparring couple Li...",Some people unravel faster than others,Locked Down,"During a COVID-19 lockdown, sparring couple Li..."
3,532865,"As WWII looms, a wealthy widow hires an amateu...",Nothing stays lost forever.,The Dig,"As WWII looms, a wealthy widow hires an amateu..."
4,775996,"In the near future, a drone pilot is sent into...",Defiant by design.,Outside the Wire,"In the near future, a drone pilot is sent into..."
...,...,...,...,...,...
356,591538,"Macbeth, the Thane of Glamis, receives a proph...",,The Tragedy of Macbeth,"Macbeth, the Thane of Glamis, receives a proph..."
357,501841,Based on the true story of First Sergeant Char...,,A Journal for Jordan,Based on the true story of First Sergeant Char...
358,673309,"The true story of Kurt Warner, who went from a...",Some dreams become destiny.,American Underdog,"The true story of Kurt Warner, who went from a..."
359,511819,A Scottish orchid farmer visiting her ill sist...,"In here, time stops.",Memoria,A Scottish orchid farmer visiting her ill sist...


In [31]:
df_2021.to_csv('desc2021.csv',index=False)

In [32]:
from google.colab import files
files.download("desc2021.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>